In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
import scipy.stats as st
import warnings
warnings.filterwarnings('ignore')

from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet




from sklearn.model_selection import cross_val_score
from sklearn import metrics

from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [5]:
data = pd.read_csv('temperature_data.csv')
data.head()

,ambient,coolant,u_d,u_q,motor_speed,torque,i_d,i_q,pm,stator_yoke,stator_tooth,stator_winding,profile_id
0,-0.752143,-1.118446,0.327935,-1.297858,-1.222428,-0.250182,1.029572,-0.245860,-2.522071,-1.831422,-2.066143,-2.018033,4
1,-0.771263,-1.117021,0.329665,-1.297686,-1.222429,-0.249133,1.029509,-0.245832,-2.522418,-1.830969,-2.064859,-2.017631,4
2,-0.782892,-1.116681,0.332771,-1.301822,-1.222428,-0.249431,1.029448,-0.245818,-2.522673,-1.830400,-2.064073,-2.017343,4
3,-0.780935,-1.116764,0.333700,-1.301852,-1.222430,-0.248636,1.032845,-0.246955,-2.521639,-1.830333,-2.063137,-2.017632,4
4,-0.774043,-1.116775,0.335206,-1.303118,-1.222429,-0.248701,1.031807,-0.246610,-2.521900,-1.830498,-2.062795,-2.018145,4


In [6]:
X = data.drop(['pm', 'stator_yoke', 'stator_tooth', 'stator_winding', 'torque'],axis=1)
X.shape

(998070, 8)

In [7]:
y = data[['pm', 'stator_yoke', 'stator_tooth', 'stator_winding', 'torque']]
y.shape

(998070, 5)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.20, shuffle= True)

In [9]:
cols = X_train.columns

In [10]:
cols

Index(['ambient', 'coolant', 'u_d', 'u_q', 'motor_speed', 'i_d', 'i_q',
       'profile_id'],
      dtype='object')

In [11]:
# Scaling the features

from sklearn.preprocessing import StandardScaler

sc_X = StandardScaler()

# fit and transform
X_trains = sc_X.fit_transform(X_train)
X_tests = sc_X.fit_transform(X_test)

In [12]:
# Scaling dataframe
X_Train = pd.DataFrame(X_trains, columns=[cols])
X_Test = pd.DataFrame(X_tests, columns=[cols])

In [13]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((798456, 8), (798456, 5), (199614, 8), (199614, 5))

In [14]:
DT = DecisionTreeRegressor(random_state=0)

In [15]:
DT.fit(X_train,y_train)
predTree = DT.predict(X_test)

In [16]:
DT_MAE =  metrics.mean_absolute_error(y_test, predTree)
DT_MAE

0.02889813660831943

In [17]:
dtr = DecisionTreeRegressor(random_state = 0)
param_grid = {'max_depth': list(range(2, 10)),
              'splitter': ['best', 'random'],
              'min_samples_leaf': list(range(1, 10)),
              'max_leaf_nodes': list(range(5, 20))}
Rand_search_dtr = RandomizedSearchCV(dtr, param_grid, cv = 5)
Rand_search_dtr.fit(X_train, y_train)
y_pred = Rand_search_dtr.predict(X_test)
r2_dtr = metrics.r2_score(y_test, y_pred)
print("The Root Mean Squared Error for Decision Tree Regressor is:", round(r2_dtr,3))


The Root Mean Squared Error for Decision Tree Regressor is: 0.672


In [24]:
import pickle

In [25]:
pickle.dump(DT,open('decision_tree_model.pkl','wb'))

In [26]:
DT = pickle.load(open('decision_tree_model.pkl','rb'))